In [ ]:
import os
import re
import shutil
import random

import numpy as np 
import pandas as pd 

from datasets import Dataset, DatasetDict, load_dataset 
from huggingface_hub import login
login("YOURTOKENHERE", add_to_git_credential=True)
import pm4py
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

In [ ]:
def secure_listdir(path, rm_list=[".ipynb_checkpoints", ".DS_Store"]):
    files_list = os.listdir(path)
    for rm_file in rm_list:
        if rm_file in files_list:
            files_list.remove(rm_file)
    return files_list

In [ ]:
data_path = "data"
data_files = secure_listdir(data_path)
data_files = [x for x in data_files if ".txt" not in x]
print(data_files)

In [ ]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
RANDOM_SEEDS = list(np.random.randint(0, 999999999, 5))
RANDOM_SEEDS

In [ ]:
DATA_COLUMNS = ["file", "trace_id", "trace_text", "event_list"]
for data_file in [x for x in data_files if x not in []]:
    log_path = os.path.join(data_path, data_file)
    tree = ET.parse(log_path)
    root = tree.getroot()
    data = []
    for child in root:
        if child.tag == "trace":
            trace_text = "\t" + str(ET.tostring(child, encoding='utf-8'), encoding="utf-8")
            trace_id = re.search(r'key\="concept\:name" value\="([A-Za-z0-9\-\.\(\)\, ]+)"', trace_text).group(1)
            event_list = [x.lstrip("\n") + "</event>" for x in re.split(r"</event>", trace_text) if "</trace>" not in x]
            event_list[-1] = event_list[-1] + "\n\t</trace>"
            data.append([data_file, trace_id, trace_text, event_list])
        if child.tag == "{http://www.xes-standard.org/}trace":
            trace_text = "\t" + str(ET.tostring(child, encoding='utf-8'), encoding="utf-8")
            trace_id = re.search(r'key\="concept\:name" value\="([A-Za-z0-9\-\.\(\)\, ]+)"', trace_text).group(1)
            event_list = [x.lstrip("\n") + "</ns0:event>" for x in re.split(r"</ns0:event>", trace_text)]
            event_list = event_list[:-1]
            event_list[-1] = event_list[-1] + "\n\t</ns0:trace>"
            data.append([data_file, trace_id, trace_text, event_list])
            raise
    data_df = pd.DataFrame(data, columns=DATA_COLUMNS)
    use_case = data_file.replace('.xes', '').lower()
    ds = Dataset.from_pandas(data_df, preserve_index=False, split="train")
    print(ds)
    ds.push_to_hub(use_case, private=True)